# Index

## 1. Introduction



## 2. Methodology


## 3. Web Scraping

### 3.1 Scraping an article


### [3.2 Scraping all article links](#3.2.-Scraping-all-article-links)


### [3.3 Putting it all together](#3.3.-Putting-it-all-together)


## 4. Text Analysis
### 4.1 Exploratory Data Analysis
### 4.2

## 1. Introduction

Back in April 2019 an extradition bill that would have allowed for criminal suspects to be extradited to mainland China under certain circumstances was introduced in Hong Kong. 
Many have said this risked exposing Hong-Kongers to unfair trials and violent treament. They also fear that the bill would give China greater influence over Hong Kong and could pose a threat to their freedom of speech.

The first protest took place on the 28th April, with tens of thousands of people marching peacefully against the extradition bill. Throughout May lawmakers attempted to come to an agreement over the bill but failed to do so on several occastions.

9th June 2019 - More than one million people took to the street to protest peacefully against the bill, however the protest turned ugly after midnight as violent protesters clash with riot police. This would mark the beginning of one of the most violent 6 months period in Hong Kong's recent history, with radical protesters clashing with police for over 50 consectutive weekends. 

After having witnessed the events myself and been exposed to several media outlets, I saw that frequently newspaper and online articles were simply not reporting the truth whether it was due to lack of accurate facts or simply being bias to one side. I decided to look into this matter further.

The ultimate aim of this project is to analysis the news articles covering the Hong Kong protest and try to determine whether an outlet is pro-democracy, pro-Beijing or neutral. Machine learning will be utilised, more specifically Neural Language Processing.

This notebook will also document my process through this project.


## 2. Methodology

## 3. Web Scraping

### 3.1. Scraping an article

Before I even begin to scape anything from the article I need to find the relevant HTML tags and content boxes for the data that I am collecting from that article. This included the article header,  summary,  timestamp,  author and the main body text. As this will be a text based analysis I did not include the article images. First let's begin by installing the modules that I will need to scrape the article.

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import csv

In [6]:
link = 'https://www.scmp.com/print/news/hong-kong/politics/article/3014737/nearly-2-million-people-take-streets-forcing-public-apology'
source = requests.get(link).text
soup = BeautifulSoup(source, 'html5lib')
#print(soup.prettify())

for article in soup.find_all('div', class_='article__wrapper wrapper'):
    title = article.h1.text
    print(f'Title: {title}')

    for summaries in article.find_all('li', class_='print-article__summary--li content--li'):
        summary = summaries.getText()
        print(f'Summary >>>> {summary}')
        
    date_published = article.find('p', class_='last-update__published published')
    date = date_published.time.getText()
    print(f'Date Published >>>> {date[10:]}')

    for main_text in article.find_all('div', class_='print-article__body article-details-type--p content--p'):
        main_text_title = main_text.getText()
        print(f'Main Text Title >>>> {main_text_title}')

    for main_text2 in article.find_all('p', class_='print-article__body article-details-type--p content--p'):
        paragraphs = main_text2.getText()
        print(f'Article Text >>>> {paragraphs}')

Title: ‘Nearly 2 million’ people take to streets, forcing public apology from Hong Kong leader Carrie Lam as suspension of controversial extradition bill fails to appease protesters
Summary >>>> Centre of city brought to a complete standstill as the masses march to chastise Lam for refusing to withdraw bill or apologise when first asked to
Summary >>>> Six hours after protesters transform Central, Wan Chai and Admiralty into a sea of black, public apology comes in the form of government statement
Date Published >>>>  1:05am, 17 Jun, 2019
Main Text Title >>>> Nearly 2 million protesters flooded the streets of Hong Kong on Sunday, organisers claimed, delivering a stunning repudiation of Chief Executive Carrie Lam Cheng Yuet-ngor’s governance and forcing a public apology out of the city’s leader over her campaign to bulldoze a controversial extradition bill through the legislature.
Article Text >>>> A day after Lam suspended her push for the bill, expecting it to defuse a crisis that has 

The above code scrapes all the content of the news article that I need, the next step would be to write them into a CSV file. But first I will need to concatanate the summary and article text into one string. This is done by appending them to an empty list, and then using the .join method. 

In [ ]:
with open('scmp_article_content_test.csv', 'w', newline='', encoding='utf-8-sig') as f:
    fieldnames = ['title', 'summary', 'date', 'main_text_title', 'paragraphs']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    article_text_list = []
    
    .
    .
    .
                for item in article:
                try:
                # unix timestamp included the millisecond so divide by 1000 is required
                    writer.writerow({'title': title, 'summary': summary,'date': date, \
                                 'main_text_title': main_text_title, 'paragraphs': article_text_conc})
                except Exception as e:
                    writer.writerow({'title': '', 'summary': '', 'date': '', 'main_text_title': '', 'paragraphs': ''})

I added the above code to the beginning and end of the article scraping codes. A couple of things to note:
    - In the `with open` statement I set the `encoding='utf-8-sig'` because the webpage was encoded with byte order mark (BOM).
    - I created the `.Dictwriter` object to map dictionaries onto output rows.
    - Added the try-except loop to handle errors, nothing will be written to that row in the CSV file if there is no content for that tag.

### 3.2. Scraping all article links

I have decided to start with scraping all the article linkes from South China Morning Post as I found them to be fairly netural and tend to cover most of the facts when available. The link I am using is https://www.scmp.com/topics/hong-kong-protests. This link contains all articles tagged to the 'Hong Kong Protest' topic.

This page uses an 'infinite scroll' feature to load more articles and in most cases an ajax request is sent, where the Request URL for the next page can easily be idenitified and used for scraping. However for this site there is no pagination used in the request URL, e.g. `https://scrapingclub.com/exercise/list_infinite_scroll/?page=2`.

In order to overcome this I had to do a little more digging,

<img src="requestURL.jpg">

### 3.3. Putting it all together

I now need to use the function written in 3.1 to read the CSV containing all the URLs and scrape all the information from each individual article.
The CSV file contains rows where there is an apostrophe, which is causing an issue when reading. To resolve this the delimiter "'" will need to be doubled for the CSV to read correctly.

20 minutes to scrape the content, around 940 rows.

## 4. Text Analysis

Once I have the content scraped it is time to do some basic EDA. For example, the number of articles posted per month and most common words in the articles (excluding stop words). First the CSV file is read into a pandas DataFrame.

In [8]:
pd.read_csv("csv_files/scmp_article_content_part1.csv", header=0)

,title,summary,date,main_text_title,paragraphs,url
0,"Protests over Yuen Long feature beer, champagn...",Largely peaceful demonstrations feature small ...,"3:55pm, 21 Feb, 2020",Peaceful lunchtime demonstrations intended to ...,At the atrium of the Landmark shopping centre ...,https://scmp.com/print/news/hong-kong/politics...
1,Coronavirus: Beijing’s top official in Hong Ko...,Luo Huining asks city’s members of China’s leg...,"11:03pm, 20 Feb, 2020",Beijing’s top envoy in Hong Kong has written t...,"Luo Huining, the new director of Beijing’s lia...",https://scmp.com/print/news/china/politics/art...
2,Anti-government protesters’ slogan spray-paint...,Popular protest refrain ‘Liberate Hong Kong; R...,"10:38pm, 19 Feb, 2020",The eight words in Chinese – “Liberate Hong Ko...,The popular slogan of the months-long anti-gov...,https://scmp.com/print/news/hong-kong/law-and-...
3,Hong Kong commerce minister warns RTHK to obse...,Commerce and Economic Development Bureau which...,"5:44pm, 18 Feb, 2020","The force earlier denied the allegation, sayin...",Hong Kong’s commerce minister has warned publi...,https://scmp.com/print/news/hong-kong/politics...
4,Videos of Hong Kong police officers dining wit...,Banquet was attended by celebrities such as Ch...,"12:18am, 18 Feb, 2020",Videos showing Hong Kong police chief Chris Ta...,The banquet on Sunday night was hosted by the ...,https://scmp.com/print/news/hong-kong/society/...
...,...,...,...,...,...,...
994,Hong Kong’s district council elections: how th...,In the first of a four-part series looking ahe...,"7:00am, 22 Oct, 2019",Hong Kong has 18 district councils made up of ...,Hong Kong’s pro-democracy camp is going all ou...,https://scmp.com/print/news/hong-kong/politics...
995,Hong Kong protests: chaos returns to streets o...,Demonstrators occupy section of Castle Peak Ro...,"12:20am, 22 Oct, 2019",Chaos returned to the streets of a northern Ho...,"Protesters, mainly masked and wearing black, o...",https://scmp.com/print/news/hong-kong/politics...
996,Hong Kong and Taiwan clash over surrender of m...,Taiwan’s Justice Minister Tsai Ching-hsiang sa...,"12:15am, 22 Oct, 2019",Both the Hong Kong and Taiwan governments have...,"Chan, who fled home to Hong Kong after the mur...",https://scmp.com/print/news/hong-kong/politics...
997,"More than 100 petrol bombs thrown, 11 MTR stat...",Fire department says protests are hindering re...,"11:10pm, 21 Oct, 2019",Both the Hong Kong and Taiwan governments have...,Anti-government protesters on Sunday threw mor...,https://scmp.com/print/news/hong-kong/politics...
